# Counterfactual with Reinforcement Learning on Adult Census

Requirements
- Python 3.10
- `pip install alibi[tensorflow] xgboost`

In [1]:
import os
import numpy as np
import pandas as pd
from copy import deepcopy
from typing import List, Tuple, Dict, Callable

import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"

import tensorflow as tf
import tensorflow.keras as keras

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from alibi.explainers import CounterfactualRLTabular, CounterfactualRL
from alibi.datasets import fetch_adult
from alibi.models.tensorflow import HeAE
from alibi.models.tensorflow import Actor, Critic
from alibi.models.tensorflow import ADULTEncoder, ADULTDecoder
from alibi.explainers.cfrl_base import Callback
from alibi.explainers.backends.cfrl_tabular import get_he_preprocessor, get_statistics, \
    get_conditional_vector, apply_category_mapping

c:\Users\xmors\Desktop\counterfactuals\alibi\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Adult Census Dataset

In [2]:
adult = fetch_adult()

categorical_names = [adult.feature_names[i] for i in adult.category_map.keys()]
categorical_ids = list(adult.category_map.keys())

numerical_names = [name for i, name in enumerate(adult.feature_names) if i not in adult.category_map.keys()]
numerical_ids = [i for i in range(len(adult.feature_names)) if i not in adult.category_map.keys()]

X, Y = adult.data, adult.target
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=13)

## Black-box model

### Train black-box classifier

In [3]:
num_transf = StandardScaler()

cat_transf = OneHotEncoder(
    categories=[range(len(x)) for x in adult.category_map.values()],
    handle_unknown="ignore"
)

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", cat_transf, categorical_ids),
        ("num", num_transf, numerical_ids),
    ],
    sparse_threshold=0
)

In [4]:
preprocessor.fit(X_train)

X_train_ohe = preprocessor.transform(X_train)
X_test_ohe = preprocessor.transform(X_test)

In [5]:
# clf = XGBClassifier(min_child_weight=0.5, max_depth=3, gamma=0.2)
# clf = LogisticRegression(C=10)
# clf = DecisionTreeClassifier(max_depth=10, min_samples_split=5)
clf = RandomForestClassifier(max_depth=15, min_samples_split=10, n_estimators=50)

clf.fit(X_train_ohe, Y_train)

RandomForestClassifier(max_depth=15, min_samples_split=10, n_estimators=50)

### Define the black-box predictor

In [7]:
predictor = lambda x: clf.predict_proba(preprocessor.transform(x))

In [8]:
acc = accuracy_score(y_true=Y_test, y_pred=predictor(X_test).argmax(axis=1))
print("Accuracy: %.3f" % acc)

Accuracy: 0.864


### Define autoencoder

In [ ]:
class HeAE(keras.Model):
    def __init__(self, encoder: keras.Model, decoder: keras.Model, **kwargs) -> None:
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def call(self, x: tf.Tensor, **kwargs):
        z = self.encoder(x)
        x_hat = self.decoder(z)
        return x_hat

In [10]:
feature_types = {"Age": int, "Capital Gain": int, "Capital Loss": int, "Hours per week": int}

heae_preprocessor, heae_inv_preprocessor = get_he_preprocessor(X=X_train,
                                                               feature_names=adult.feature_names,
                                                               category_map=adult.category_map,
                                                               feature_types=feature_types)

trainset_input = heae_preprocessor(X_train).astype(np.float32)
trainset_outputs = {
    "output_1": trainset_input[:, :len(numerical_ids)]
}

for i, cat_id in enumerate(categorical_ids):
    trainset_outputs.update({
        f"output_{i+2}": X_train[:, cat_id]
    })

trainset = tf.data.Dataset.from_tensor_slices((trainset_input, trainset_outputs))
trainset = trainset.shuffle(1024).batch(128, drop_remainder=True)

### Train autoencoder

In [11]:
heae_path = os.path.join("models", "alibi", "ADULT_autoencoder")
if not os.path.exists(heae_path):
    os.makedirs(heae_path)

EPOCHS = 50              # epochs to train the autoencoder
HIDDEN_DIM = 128         # hidden dimension of the autoencoder
LATENT_DIM = 15          # define latent dimension

OUTPUT_DIMS = [len(numerical_ids)]
OUTPUT_DIMS += [len(adult.category_map[cat_id]) for cat_id in categorical_ids]

heae = HeAE(encoder=ADULTEncoder(hidden_dim=HIDDEN_DIM, latent_dim=LATENT_DIM),
            decoder=ADULTDecoder(hidden_dim=HIDDEN_DIM, output_dims=OUTPUT_DIMS))

he_loss = [keras.losses.MeanSquaredError()]
he_loss_weights = [1.]

for i in range(len(categorical_names)):
    he_loss.append(keras.losses.SparseCategoricalCrossentropy(from_logits=True))
    he_loss_weights.append(1./len(categorical_names))

metrics = {}
for i, cat_name in enumerate(categorical_names):
    metrics.update({f"output_{i+2}": keras.metrics.SparseCategoricalAccuracy()})

heae.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
             loss=he_loss,
             loss_weights=he_loss_weights,
             metrics=metrics)

if len(os.listdir(heae_path)) == 0:
    heae.fit(trainset, epochs=EPOCHS)
    heae.save(heae_path, save_format="tf")
else:
    heae = keras.models.load_model(heae_path, compile=False)

Epoch 1/50
203/203 [==============================] - 2s 2ms/step - loss: 1.1108 - output_1_loss: 0.2215 - output_2_loss: 1.1230 - output_3_loss: 1.1727 - output_4_loss: 0.6519 - output_5_loss: 1.6178 - output_6_loss: 0.9538 - output_7_loss: 0.6090 - output_8_loss: 0.3023 - output_9_loss: 0.6840 - output_2_sparse_categorical_accuracy: 0.6781 - output_3_sparse_categorical_accuracy: 0.5974 - output_4_sparse_categorical_accuracy: 0.7675 - output_5_sparse_categorical_accuracy: 0.4355 - output_6_sparse_categorical_accuracy: 0.6824 - output_7_sparse_categorical_accuracy: 0.8264 - output_8_sparse_categorical_accuracy: 0.8766 - output_9_sparse_categorical_accuracy: 0.8702
Epoch 2/50
203/203 [==============================] - 0s 2ms/step - loss: 0.3979 - output_1_loss: 0.0429 - output_2_loss: 0.5515 - output_3_loss: 0.4727 - output_4_loss: 0.2066 - output_5_loss: 0.6344 - output_6_loss: 0.3108 - output_7_loss: 0.2778 - output_8_loss: 0.0670 - output_9_loss: 0.3195 - output_2_sparse_categorical_

INFO:tensorflow:Assets written to: models\alibi\ADULT_autoencoder\assets


## Generate counterfactuals with reinforcement learning

In [12]:
COEFF_SPARSITY = 0.5               # sparisty coefficient
COEFF_CONSISTENCY = 0.5            # consisteny coefficient
TRAIN_STEPS = 10000                # number of training steps -> consider increasing the number of steps
BATCH_SIZE = 100                   # batch size

### Define attributes and constraints

In [13]:
immutable_features = ['Marital Status', 'Relationship', 'Race', 'Sex']

ranges = {'Age': [0.0, 1.0]}

### Define and fit the explainer

In [14]:
explainer = CounterfactualRLTabular(predictor=predictor,
                                    encoder=heae.encoder,
                                    decoder=heae.decoder,
                                    latent_dim=LATENT_DIM,
                                    encoder_preprocessor=heae_preprocessor,
                                    decoder_inv_preprocessor=heae_inv_preprocessor,
                                    coeff_sparsity=COEFF_SPARSITY,
                                    coeff_consistency=COEFF_CONSISTENCY,
                                    category_map=adult.category_map,
                                    feature_names=adult.feature_names,
                                    ranges=ranges,
                                    immutable_features=immutable_features,
                                    train_steps=TRAIN_STEPS,
                                    batch_size=BATCH_SIZE,
                                    backend="tensorflow")

In [15]:
explainer = explainer.fit(X=X_train)

100%|██████████| 10000/10000 [06:58<00:00, 23.91it/s]


### Test the explainer

In [16]:
X_positive = X_test[np.argmax(predictor(X_test), axis=1) == 1]

X = X_positive[:1000]
Y_t = np.array([0])
C = [{"Age": [0, 20], "Workclass": ["State-gov", "?", "Local-gov"]}]

In [17]:
explanation = explainer.explain(X, Y_t, C)

100%|██████████| 10/10 [00:00<00:00, 37.40it/s]


In [18]:
orig = np.concatenate(
    [explanation.data['orig']['X'], explanation.data['orig']['class']],
    axis=1
)

cf = np.concatenate(
    [explanation.data['cf']['X'], explanation.data['cf']['class']],
    axis=1
)

feature_names = adult.feature_names + ["Label"]
category_map = deepcopy(adult.category_map)
category_map.update({feature_names.index("Label"): adult.target_names})

orig_pd = pd.DataFrame(
    apply_category_mapping(orig, category_map),
    columns=feature_names
)

cf_pd = pd.DataFrame(
    apply_category_mapping(cf, category_map),
    columns=feature_names
)

c:\Users\xmors\Desktop\counterfactuals\alibi\lib\site-packages\alibi\explainers\backends\cfrl_tabular.py:877: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  pd_X[key].replace(range(len(category_map[key])), category_map[key], inplace=True)


In [19]:
orig_pd.head(n=10)

,Age,Workclass,Education,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country,Label
0,60,Private,High School grad,Married,Blue-Collar,Husband,White,Male,7298,0,40,United-States,>50K
1,35,Private,High School grad,Married,White-Collar,Husband,White,Male,7688,0,50,United-States,>50K
2,39,State-gov,Masters,Married,Professional,Wife,White,Female,5178,0,38,United-States,>50K
3,44,Self-emp-inc,High School grad,Married,Sales,Husband,White,Male,0,0,50,United-States,>50K
4,39,Private,Bachelors,Separated,White-Collar,Not-in-family,White,Female,13550,0,50,United-States,>50K
5,45,Private,High School grad,Married,Blue-Collar,Husband,White,Male,0,1902,40,?,>50K
6,50,Private,Bachelors,Married,Professional,Husband,White,Male,0,0,50,United-States,>50K
7,29,Private,Bachelors,Married,White-Collar,Wife,White,Female,0,0,50,United-States,>50K
8,47,Private,Bachelors,Married,Professional,Husband,White,Male,0,0,50,United-States,>50K
9,35,Private,Bachelors,Married,White-Collar,Husband,White,Male,0,0,70,United-States,>50K


In [20]:
cf_pd.head(n=10)

,Age,Workclass,Education,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country,Label
0,60,Private,High School grad,Married,Blue-Collar,Husband,White,Male,218,2,40,United-States,<=50K
1,35,Private,High School grad,Married,Blue-Collar,Husband,White,Male,117,3,50,United-States,<=50K
2,39,State-gov,High School grad,Married,Blue-Collar,Wife,White,Female,204,1,38,United-States,<=50K
3,44,Self-emp-inc,High School grad,Married,Service,Husband,White,Male,1,0,50,United-States,<=50K
4,39,Private,Bachelors,Separated,White-Collar,Not-in-family,White,Female,272,0,48,United-States,<=50K
5,45,Private,High School grad,Married,Blue-Collar,Husband,White,Male,150,1919,41,South-America,>50K
6,50,Private,Dropout,Married,Blue-Collar,Husband,White,Male,52,0,50,United-States,<=50K
7,29,Private,Dropout,Married,Blue-Collar,Wife,White,Female,254,0,49,United-States,<=50K
8,47,Private,Dropout,Married,Blue-Collar,Husband,White,Male,58,0,50,United-States,<=50K
9,35,Private,Dropout,Married,Blue-Collar,Husband,White,Male,3,0,70,United-States,<=50K


### Test with `diversity=True`

In [21]:
X = X_positive[0].reshape(1, -1)
explanation = explainer.explain(X=X, Y_t=Y_t, C=C, diversity=True, num_samples=100, batch_size=10)

11it [00:00, 33.59it/s]


In [22]:
orig = np.concatenate(
    [explanation.data['orig']['X'], explanation.data['orig']['class']],
    axis=1
)

cf = np.concatenate(
    [explanation.data['cf']['X'], explanation.data['cf']['class']],
    axis=1
)

orig_pd = pd.DataFrame(
    apply_category_mapping(orig, category_map),
    columns=feature_names,
)

cf_pd = pd.DataFrame(
    apply_category_mapping(cf, category_map),
    columns=feature_names,
)

c:\Users\xmors\Desktop\counterfactuals\alibi\lib\site-packages\alibi\explainers\backends\cfrl_tabular.py:877: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  pd_X[key].replace(range(len(category_map[key])), category_map[key], inplace=True)


In [23]:
orig_pd.head(n=5)

,Age,Workclass,Education,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country,Label
0,60,Private,High School grad,Married,Blue-Collar,Husband,White,Male,7298,0,40,United-States,>50K


In [24]:
cf_pd.head(n=5)

,Age,Workclass,Education,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country,Label
0,60,Private,High School grad,Married,Blue-Collar,Husband,White,Male,6,0,41,United-States,<=50K
1,60,Private,High School grad,Married,Blue-Collar,Husband,White,Male,13,0,41,United-States,<=50K
2,60,Private,High School grad,Married,Blue-Collar,Husband,White,Male,23,0,41,United-States,<=50K
3,60,Private,High School grad,Married,Blue-Collar,Husband,White,Male,36,0,41,United-States,<=50K
4,60,Private,High School grad,Married,Blue-Collar,Husband,White,Male,38,0,40,United-States,<=50K
